[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/motion_estimation/blob/main/src/motion_estimation/full_search_dense_ME.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/vicente-gonzalez-ruiz/motion_estimation/blob/main/src/motion_estimation/full_search_dense_ME.ipynb)

# Optical flow image denoising using the optical flow


In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !apt update
    !apt install imagemagick
    !apt install cm-super
    !apt install dvipng
    !apt install bc
    !apt install texlive-latex-extra
    !apt install texlive-fonts-recommended

In [2]:
!which python

/shared/PEs/opencv/bin/python


In [3]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Uninstalling pip-22.0.2:
      Successfully uninstalled pip-22.0.2


In [4]:
import numpy as np

try:
    from scipy import ndimage
except:
    !pip install scipy
    from scipy import ndimage

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab

plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{amsmath}" #for \text command
   
%matplotlib inline

In [6]:
try:
    from color_transforms import YCoCg as YUV
except:
    !pip install "color_transforms @ git+https://github.com/vicente-gonzalez-ruiz/color_transforms"
    from color_transforms import YCoCg as YUV

In [7]:
try:
    from image_IO import image_1 as gray_image
    from image_IO import image_3 as RGB_image
except:
    !pip install "image_IO @ git+https://github.com/vicente-gonzalez-ruiz/image_IO"
    from image_IO import image_1 as gray_image
    from image_IO import image_3 as RGB_image

  Cloning https://github.com/vicente-gonzalez-ruiz/image_IO to /tmp/pip-install-3xfs0zal/image-io_e15216953ec24607b8724e366d56610d
  Running command git clone --filter=blob:none --quiet https://github.com/vicente-gonzalez-ruiz/image_IO /tmp/pip-install-3xfs0zal/image-io_e15216953ec24607b8724e366d56610d
  Resolved https://github.com/vicente-gonzalez-ruiz/image_IO to commit 5dff57f270436b2f3de403e9ae1eeb2ee7be5712
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for image_IO: filename=image_IO-0.0.1-py3-none-any.whl size=4699 sha256=99f27dc8b543b91370ff542b62dcfe35f67ecb174bea8ebda51788b97fe98f7a
  Stored in directory: /tmp/pip-ephem-wheel-cache-pe3n2add/wheels/20/90/b9/f15d31b7a36249b9fb7b8ab1ad46eb55e45dbdb02ab17bdb7a
Successfully built image_IO


In [8]:
try:
    from information_theory import information
    #from information_theory import distortion
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    from information_theory import information
    #from information_theory import distortion

In [9]:
import image_denoising

AttributeError: module 'motion_estimation.farneback' has no attribute 'Estimator_in_CPU'

In [ ]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

In [ ]:
import cv2
import random

In [ ]:
import urllib
try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

fn = "http://www.hpca.ual.es/~vruiz/images/lena.png"
req = urllib.request.Request(fn, method='HEAD')
f = urllib.request.urlopen(req)
img = skimage_io.imread(fn)
RGB_image.show(img,"")

In [ ]:
row, col, ch = img.shape
mean = 0
var = 1000
sigma = var**0.5
noise = np.random.normal(mean, sigma, (row, col, ch))
print(np.max(noise))
noisy_image = np.clip(a=img.astype(np.float32) + noise, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
RGB_image.show(noisy_image,"")

In [ ]:
denoised_image = cv2.fastNlMeansDenoisingColored(noisy_image, None, 11, 6, 7, 21)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 24))
axs[0].imshow(noisy_image.astype(np.uint8))
axs[0].set_title("noisy")
axs[1].imshow(denoised_image.astype(np.uint8))
axs[1].set_title("denoised")
axs[2].imshow(img.astype(np.uint8))
axs[2].set_title("GT")
fig.tight_layout()
plt.show()

In [ ]:
denoised_image, _ = image_denoising.OF_gaussian.filter_RGB_image(
    noisy_image,
    sigma_kernel=2.5,
    N_iters=1,
    l=3,
    w=9,
    sigma_OF=2.5,
    GT=img
)

In [ ]:
np.amax(_)

In [ ]:
np.argmax(_)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 24))
axs[0].imshow(noisy_image.astype(np.uint8))
axs[0].set_title("noisy")
axs[1].imshow(denoised_image.astype(np.uint8))
axs[1].set_title("denoised")
axs[2].imshow(img.astype(np.uint8))
axs[2].set_title("GT")
fig.tight_layout()
plt.show()

In [ ]:
image_denoiser = image_denoising.OF_random.Filter_Color_Image(levels=5, window_side=15, poly_sigma=0.5)

In [ ]:
RD_iters = 50
RD_sigma = 1.0
RD_mean = 0.0

In [ ]:
denoised_image, _ = image_denoiser.filter(
    noisy_image,
    RD_iters,
    RD_sigma,
    GT=img
)

In [ ]:
np.amax(_)

In [ ]:
np.argmax(_)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 24))
axs[0].imshow(noisy_image.astype(np.uint8))
axs[0].set_title("noisy")
axs[1].imshow(denoised_image.astype(np.uint8))
axs[1].set_title("denoised")
axs[2].imshow(img.astype(np.uint8))
axs[2].set_title("GT")
fig.tight_layout()
plt.show()

In [ ]:
image_denoiser = image_denoising.OF_random.Filter_Color_Image(levels=5, window_side=5, poly_sigma=0.5)

In [ ]:
RD_iters = 50
RD_sigma = 1.0
RD_mean = 0.0

In [ ]:
denoised_image, _ = image_denoiser.filter(
    noisy_image,
    RD_iters,
    RD_sigma,
    GT=img
)

In [ ]:
np.amax(_)

In [ ]:
np.argmax(_)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 24))
axs[0].imshow(noisy_image.astype(np.uint8))
axs[0].set_title("noisy")
axs[1].imshow(denoised_image.astype(np.uint8))
axs[1].set_title("denoised")
axs[2].imshow(img.astype(np.uint8))
axs[2].set_title("GT")
fig.tight_layout()
plt.show()

In [ ]:
image_denoiser = image_denoising.OF_random.Filter_Color_Image(levels=5, window_side=7, poly_sigma=0.3)

In [ ]:
RD_iters = 50
RD_sigma = 1.0
RD_mean = 0.0

In [ ]:
denoised_image, _ = image_denoiser.filter(
    noisy_image,
    RD_iters,
    RD_sigma,
    GT=img
)

In [ ]:
np.amax(_)

In [ ]:
np.argmax(_)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 24))
axs[0].imshow(noisy_image.astype(np.uint8))
axs[0].set_title("noisy")
axs[1].imshow(denoised_image.astype(np.uint8))
axs[1].set_title("denoised")
axs[2].imshow(img.astype(np.uint8))
axs[2].set_title("GT")
fig.tight_layout()
plt.show()

In [ ]:
image_denoiser = image_denoising.OF_random.Filter_Color_Image(levels=5, window_side=7, poly_sigma=1.5)

In [ ]:
RD_iters = 50
RD_sigma = 1.0
RD_mean = 0.0

In [ ]:
denoised_image, _ = image_denoiser.filter(
    noisy_image,
    RD_iters,
    RD_sigma,
    GT=img
)

In [ ]:
np.amax(_)

In [ ]:
np.argmax(_)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 24))
axs[0].imshow(noisy_image.astype(np.uint8))
axs[0].set_title("noisy")
axs[1].imshow(denoised_image.astype(np.uint8))
axs[1].set_title("denoised")
axs[2].imshow(img.astype(np.uint8))
axs[2].set_title("GT")
fig.tight_layout()
plt.show()

In [ ]:
def randomize(image, max_distance_x=1, max_distance_y=1):
    height, width, _ = image.shape
    x_coords, y_coords = np.meshgrid(range(width), range(height)) # Create a grid of coordinates
    flattened_x_coords = x_coords.flatten()
    flattened_y_coords = y_coords.flatten()
    displacements_x = np.random.randint(-max_distance_x, max_distance_x + 1, flattened_x_coords.shape)
    displacements_y = np.random.randint(-max_distance_y, max_distance_y + 1, flattened_y_coords.shape)
    randomized_x_coords = flattened_x_coords + displacements_x
    randomized_y_coords = flattened_y_coords + displacements_y
    randomized_x_coords = np.mod(randomized_x_coords, width) # Use periodic extension to handle border pixels
    randomized_y_coords = np.mod(randomized_y_coords, height)
    randomized_image = np.empty_like(image)
    randomized_image[...] = image
    randomized_image[randomized_y_coords,
                     randomized_x_coords, :] = image[flattened_y_coords,
                                                     flattened_x_coords, :]
    return randomized_image

def get_flow_to_project_A_to_B(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    print(A.shape, B.shape, A.dtype, B.dtype)
    flow = cv2.calcOpticalFlowFarneback(prev=B, next=A, flow=prev_flow,
                                        pyr_scale=0.5, levels=l, winsize=w,
                                        iterations=3, poly_n=5, poly_sigma=sigma,
                                        flags=0)
    #flow[...] = 0.0
    print(np.average(np.abs(flow)), l, w, sigma)
    return flow

def project(image, flow):
    height, width = flow.shape[:2]
    map_x = np.tile(np.arange(width), (height, 1))
    map_y = np.swapaxes(np.tile(np.arange(height), (width, 1)), 0, 1)
    map_xy = (flow + np.dstack((map_x, map_y))).astype('float32')
    warped_image = cv2.remap(image, map_xy, None,
                             #interpolation=cv2.INTER_LANCZOS4, #INTER_LINEAR,
                             interpolation=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_REPLICATE)
    return warped_image

def project_A_to_B(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    A_luma = YUV.from_RGB(A.astype(np.int16))[..., 0]
    B_luma = YUV.from_RGB(B.astype(np.int16))[..., 0]
    flow = get_flow_to_project_A_to_B(A_luma, B_luma, l, w, prev_flow, sigma)
    return project(A, flow)

In [ ]:
'''1. denoised_image = 0
2. for i in range(iters):
    1. randomized_noisy_image = randomize(noisy_image)
    2. randomized_and_compensated_noisy_image = project_A_to_B(A=randomized_noisy_image, B=noisy_image)
    3. average_image += (noisy_image + randomized_and_compensated_noisy_image)/2
    4. denoised_image += average_image
3. denoised_image /= iters
'''
'''
iters = 20
denoised_image = np.empty_like(noisy_image, dtype=np.float32)
denoised_image[...] = noisy_image
for i in range(iters):
    randomized_noisy_image = randomize_image(denoised_image)
    OF_compensated_randomized_noisy_image = project_A_to_B(A=randomized_noisy_image, B=denoised_image,
                                                          l=2, w=33, sigma=3.5)
    denoised_image = (denoised_image + OF_compensated_randomized_noisy_image)/2
    plt.imshow(denoised_image.astype(np.uint8))
    plt.show()
'''
acc_image = np.zeros_like(noisy_image, dtype=np.float32)
acc_image[...] = img
iters = 100
for i in range(iters):
    denoised_image = acc_image/(i+1)
    #randomized_noisy_image = randomize(noisy_image, max_distance_x=50, max_distance_y=50).astype(np.float32)
    randomized_noisy_image = randomize(img, max_distance_x=10, max_distance_y=10).astype(np.float32)
    #randomized_and_compensated_noisy_image = project_A_to_B(A=randomized_noisy_image, B=noisy_image, l=5, w=15, sigma=1.5)
    randomized_and_compensated_noisy_image = project_A_to_B(B=randomized_noisy_image, A=denoised_image, l=5, w=15, sigma=1.5)
    #randomized_and_compensated_noisy_image = randomized_noisy_image
    #plt.imshow(randomized_noisy_image.astype(np.uint8))
    #plt.show()
    acc_image += randomized_and_compensated_noisy_image
denoised_image = acc_image/(iters)
print(np.max(denoised_image))

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 24))
axs[0].imshow(noisy_image.astype(np.uint8))
axs[0].set_title("noisy")
axs[1].imshow(denoised_image.astype(np.uint8))
axs[1].set_title("denoised")
axs[2].imshow(img.astype(np.uint8))
axs[2].set_title("GT")
fig.tight_layout()
plt.show()

In [ ]:
RGB_image.show(denoised_image.astype(np.uint8),"")

In [ ]:
acc_image = np.zeros_like(noisy_image, dtype=np.float32)
acc_image[...] = noisy_image
iters = 50
for i in range(iters):
    denoised_image = acc_image/(i+1)
    #randomized_noisy_image = randomize(noisy_image, max_distance_x=50, max_distance_y=50).astype(np.float32)
    randomized_noisy_image = randomize(noisy_image, max_distance_x=9, max_distance_y=9).astype(np.float32)
    #randomized_and_compensated_noisy_image = project_A_to_B(A=randomized_noisy_image, B=noisy_image, l=5, w=15, sigma=1.5)
    randomized_and_compensated_noisy_image = project_A_to_B(B=randomized_noisy_image, A=denoised_image, l=3, w=2, sigma=0.3)
    #randomized_and_compensated_noisy_image = randomized_noisy_image
    #plt.imshow(randomized_noisy_image.astype(np.uint8))
    #plt.show()
    acc_image += randomized_and_compensated_noisy_image
denoised_image = acc_image/(iters + 1)

In [ ]:
RGB_image.show(denoised_image.astype(np.uint8),"")

In [ ]:
input()

In [ ]:
acc_image = np.zeros_like(img, dtype=np.float32)
acc_image[...] = img#noisy_image
iters = 50
for i in range(iters):
    denoised_image = acc_image/(i+1)
    #randomized_noisy_image = randomize(noisy_image, max_distance_x=50, max_distance_y=50).astype(np.float32)
    randomized_noisy_image = randomize(img, max_distance_x=19, max_distance_y=19).astype(np.float32)
    #randomized_and_compensated_noisy_image = project_A_to_B(A=randomized_noisy_image, B=noisy_image, l=5, w=15, sigma=1.5)
    randomized_and_compensated_noisy_image = project_A_to_B(B=randomized_noisy_image, A=denoised_image, l=3, w=2, sigma=0.3)
    #randomized_and_compensated_noisy_image = randomized_noisy_image
    #plt.imshow(randomized_noisy_image.astype(np.uint8))
    #plt.show()
    acc_image += randomized_and_compensated_noisy_image
denoised_image = acc_image/(iters + 1)

In [ ]:
RGB_image.show(denoised_image.astype(np.uint8),"")

In [ ]:
def randomize(image, max_distance_y=9, max_distance_x=9):
    height, width, _ = image.shape
    x_coords, y_coords = np.meshgrid(range(width), range(height)) # Create a grid of coordinates
    flattened_x_coords = x_coords.flatten()
    flattened_y_coords = y_coords.flatten()
    displacements_x = np.random.randint(-max_distance_x, max_distance_x + 1, flattened_x_coords.shape)
    displacements_y = np.random.randint(-max_distance_y, max_distance_y + 1, flattened_y_coords.shape)
    randomized_x_coords = flattened_x_coords + displacements_x
    randomized_y_coords = flattened_y_coords + displacements_y
    randomized_x_coords = np.mod(randomized_x_coords, width) # Use periodic extension to handle border pixels
    randomized_y_coords = np.mod(randomized_y_coords, height)
    randomized_image = np.empty_like(image)
    randomized_image[...] = image
    randomized_image[randomized_y_coords,
                     randomized_x_coords, :] = image[flattened_y_coords,
                                                     flattened_x_coords, :]
    return randomized_image

def get_flow_to_project_A_to_B(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    flow = cv2.calcOpticalFlowFarneback(prev=B, next=A, flow=prev_flow,
                                        pyr_scale=0.5, levels=l, winsize=w,
                                        iterations=3, poly_n=5, poly_sigma=sigma,
                                        flags=0)
    #flow[...] = 0.0
    print(np.average(np.abs(flow)), l, w, sigma, end=' ')
    return flow

def project(image, flow):
    height, width = flow.shape[:2]
    map_x = np.tile(np.arange(width), (height, 1))
    map_y = np.swapaxes(np.tile(np.arange(height), (width, 1)), 0, 1)
    map_xy = (flow + np.dstack((map_x, map_y))).astype('float32')
    warped_image = cv2.remap(image, map_xy, None,
                             #interpolation=cv2.INTER_LANCZOS4, #INTER_LINEAR,
                             interpolation=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_REPLICATE)
    return warped_image

def warp_A_to_B(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    A_luma = YUV.from_RGB(A.astype(np.int16))[..., 0]
    B_luma = YUV.from_RGB(B.astype(np.int16))[..., 0]
    flow = get_flow_to_project_A_to_B(A_luma, B_luma, l, w, prev_flow, sigma)
    return project(A, flow)

def denoise(noisy_image, iters=50, max_distance_y=9, max_distance_x=9, l=3, w=2, sigma=0.3):
    acc_image = np.zeros_like(noisy_image, dtype=np.float32)
    acc_image[...] = noisy_image
    for i in range(iters):
        print(i, end=' ')
        denoised_image = acc_image/(i+1)
        randomized_noisy_image = randomize(noisy_image, max_distance_y=9, max_distance_x=9).astype(np.float32)
        randomized_and_compensated_noisy_image = warp_A_to_B(B=randomized_noisy_image, A=denoised_image, l=3, w=2, sigma=0.3)
        #randomized_and_compensated_noisy_image = randomized_noisy_image
        #plt.imshow(randomized_noisy_image.astype(np.uint8))
        #plt.show()
        acc_image += randomized_and_compensated_noisy_image
    denoised_image = acc_image/(iters + 1)
    print()
    return denoised_image

In [ ]:
RGB_image.show(
    denoise(
        img,
        iters=50,
        max_distance_y=19, max_distance_x=19,
        l=3, w=2, sigma=0.3).astype(np.uint8),"")

In [ ]:
def randomize(image, max_distance_y=9, max_distance_x=9):
    print("randomize:", max_distance_y, max_distance_x)
    height, width, _ = image.shape
    x_coords, y_coords = np.meshgrid(range(width), range(height)) # Create a grid of coordinates
    flattened_x_coords = x_coords.flatten()
    flattened_y_coords = y_coords.flatten()
    displacements_x = np.random.randint(-max_distance_x, max_distance_x + 1, flattened_x_coords.shape)
    displacements_y = np.random.randint(-max_distance_y, max_distance_y + 1, flattened_y_coords.shape)
    randomized_x_coords = flattened_x_coords + displacements_x
    randomized_y_coords = flattened_y_coords + displacements_y
    randomized_x_coords = np.mod(randomized_x_coords, width) # Use periodic extension to handle border pixels
    randomized_y_coords = np.mod(randomized_y_coords, height)
    randomized_image = np.empty_like(image)
    randomized_image[...] = image
    randomized_image[randomized_y_coords,
                     randomized_x_coords, :] = image[flattened_y_coords,
                                                     flattened_x_coords, :]
    return randomized_image

def get_flow_to_project_A_to_B(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    flow = cv2.calcOpticalFlowFarneback(prev=A, next=B, flow=prev_flow,
                                        pyr_scale=0.5, levels=l, winsize=w,
                                        iterations=3, poly_n=5, poly_sigma=sigma,
                                        flags=0 #cv2.OPTFLOW_FARNEBACK_GAUSSIAN
                                       )
    #flow[...] = 0.0
    print(np.average(np.abs(flow)), l, w, sigma, end=' ')
    return flow

def project(image, flow):
    height, width = flow.shape[:2]
    map_x = np.tile(np.arange(width), (height, 1))
    map_y = np.swapaxes(np.tile(np.arange(height), (width, 1)), 0, 1)
    map_xy = (flow + np.dstack((map_x, map_y))).astype('float32')
    warped_image = cv2.remap(image, map_xy, None,
                             #interpolation=cv2.INTER_LANCZOS4, #INTER_LINEAR,
                             interpolation=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_REPLICATE)
    return warped_image

def warp_B_to_A(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    A_luma = YUV.from_RGB(A.astype(np.int16))[..., 0]
    B_luma = YUV.from_RGB(B.astype(np.int16))[..., 0]
    flow = get_flow_to_project_A_to_B(A_luma, B_luma, l, w, prev_flow, sigma)
    return project(B, flow)

def denoise(noisy_image, iters=50, max_distance_y=9, max_distance_x=9, l=3, w=2, sigma=0.3):
    print(iters, max_distance_y, max_distance_x, l, w, sigma)
    acc_image = np.zeros_like(noisy_image, dtype=np.float32)
    acc_image[...] = noisy_image
    for i in range(iters):
        print(i, end=' ')
        denoised_image = acc_image/(i+1)
        randomized_noisy_image = randomize(noisy_image, max_distance_y, max_distance_x).astype(np.float32)
        randomized_and_compensated_noisy_image = warp_B_to_A(A=randomized_noisy_image, B=denoised_image, l=l, w=w, sigma=sigma)
        #randomized_and_compensated_noisy_image = randomized_noisy_image
        plt.imshow(randomized_noisy_image.astype(np.uint8))
        plt.show()
        acc_image += randomized_and_compensated_noisy_image
    denoised_image = acc_image/(iters + 1)
    print()
    return denoised_image

In [ ]:
RGB_image.show(
    denoise(
        img,
        iters=10,
        max_distance_y=10, max_distance_x=10,
        l=5, w=9, sigma=0.3).astype(np.uint8),"")

In [ ]:
def randomize(image, max_distance_y=9, max_distance_x=9):
    print("randomize:", max_distance_y, max_distance_x)
    height, width = image.shape[:2]
    x_coords, y_coords = np.meshgrid(range(width), range(height)) # Create a grid of coordinates
    flattened_x_coords = x_coords.flatten()
    flattened_y_coords = y_coords.flatten()
    displacements_x = np.random.randint(-max_distance_x, max_distance_x + 1, flattened_x_coords.shape)
    displacements_y = np.random.randint(-max_distance_y, max_distance_y + 1, flattened_y_coords.shape)
    randomized_x_coords = flattened_x_coords + displacements_x
    randomized_y_coords = flattened_y_coords + displacements_y
    randomized_x_coords = np.mod(randomized_x_coords, width) # Use periodic extension to handle border pixels
    randomized_y_coords = np.mod(randomized_y_coords, height)
    randomized_image = np.empty_like(image)
    randomized_image[...] = image
    randomized_image[randomized_y_coords,
                     randomized_x_coords] = image[flattened_y_coords,
                                                  flattened_x_coords]
    return randomized_image

def get_flow_to_project_A_to_B(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    # projection(next, flow) ~ prev
    flow = cv2.calcOpticalFlowFarneback(prev=A, next=B, flow=prev_flow,
                                        pyr_scale=0.5, levels=l, winsize=w,
                                        iterations=3, poly_n=5, poly_sigma=sigma,
                                        flags=0 #cv2.OPTFLOW_FARNEBACK_GAUSSIAN
                                       )
    #flow[...] = 0.0
    print(np.average(np.abs(flow)), l, w, sigma, end=' ')
    '''
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Normalize the magnitude to the range [0, 255]
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    # Convert angle to degrees for visualization
    angle = np.rad2deg(angle)

    # Create HSV image
    hsv = np.zeros((A.shape[0], A.shape[1], 3), dtype=np.uint8)
    hsv[..., 0] = angle / 2
    hsv[..., 1] = 255
    hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    # Convert HSV to BGR for visualization
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Display the flow field
    plt.imshow(rgb)
    plt.show()
    '''
    return flow

def project(image, flow):
    height, width = flow.shape[:2]
    map_x = np.tile(np.arange(width), (height, 1))
    map_y = np.swapaxes(np.tile(np.arange(height), (width, 1)), 0, 1)
    map_xy = (flow + np.dstack((map_x, map_y))).astype('float32')
    warped_image = cv2.remap(image, map_xy, None,
                             #interpolation=cv2.INTER_LANCZOS4, #INTER_LINEAR,
                             interpolation=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_REPLICATE)
    return warped_image

def RGB_warp_B_to_A(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    A_luma = YUV.from_RGB(A.astype(np.int16))[..., 0]
    B_luma = YUV.from_RGB(B.astype(np.int16))[..., 0]
    flow = get_flow_to_project_A_to_B(A_luma, B_luma, l, w, prev_flow, sigma)
    return project(B, flow)

def warp_B_to_A(A, B, l=3, w=15, prev_flow=None, sigma=1.5):
    flow = get_flow_to_project_A_to_B(A, B, l, w, prev_flow, sigma)
    return project(B, flow)

def denoise(warp_B_to_A, noisy_image, iters=50, max_distance_y=9, max_distance_x=9, l=3, w=2, sigma=0.3):
    print(iters, max_distance_y, max_distance_x, l, w, sigma)
    acc_image = np.zeros_like(noisy_image, dtype=np.float32)
    acc_image[...] = noisy_image
    for i in range(iters):
        print(i, end=' ')
        denoised_image = acc_image/(i+1)
        randomized_noisy_image = randomize(noisy_image, max_distance_y, max_distance_x).astype(np.float32)
        #randomized_noisy_image = randomize(denoised_image, max_distance_y, max_distance_x).astype(np.float32)
        #randomized_noisy_image = noisy_image
        #randomized_and_compensated_noisy_image = warp_B_to_A(B=randomized_noisy_image, A=noisy_image, l=l, w=w, sigma=sigma)
        #randomized_and_compensated_noisy_image = warp_B_to_A(B=randomized_noisy_image, A=denoised_image, l=l, w=w, sigma=sigma)
        randomized_and_compensated_noisy_image = warp_B_to_A(A=randomized_noisy_image, B=denoised_image, l=l, w=w, sigma=sigma)
        #randomized_and_compensated_noisy_image = randomized_noisy_image
        plt.imshow(randomized_and_compensated_noisy_image.astype(np.uint8))
        plt.show()
        acc_image += randomized_and_compensated_noisy_image
    denoised_image = acc_image/(iters + 1)
    print()
    return denoised_image

In [ ]:
denoised_img = denoise(RGB_warp_B_to_A, noisy_image,
                       iters=10, max_distance_y=5, max_distance_x=5, l=2, w=9, sigma=1.25)

In [ ]:
denoised_img = denoise(RGB_warp_B_to_A, noisy_image,
                       iters=100, max_distance_y=15, max_distance_x=15, l=3, w=9, sigma=1.25)

In [ ]:
RGB_image.show(denoised_img.astype(np.uint8))

In [ ]:
denoised_img = denoise(RGB_warp_B_to_A, noisy_image,
                       iters=100, max_distance_y=10, max_distance_x=10, l=3, w=9, sigma=1.25)

In [ ]:
RGB_image.show(denoised_img.astype(np.uint8))